# Keep the CLS et SEP tokens for the calculus of the distribution

In this notebook we will look at how the CLS token is constructed, we will only look at the attention on the CLS tokens

In [59]:
%load_ext autoreload
%autoreload 2

import os
from os import path
# set the repository to the git repository
cwd = os.getcwd().split(os.path.sep)
while cwd[-1] != "stage_4_gm":
    os.chdir("..")
    cwd = os.getcwd().split(os.path.sep)
print(">> the git rep : ", end="")
print(os.getcwd())

# the folder where we will save our data
plots_folder = os.path.join(os.getcwd(), '.cache', 'plots')
graph_folder = path.join(plots_folder, "sep_cls_study")
if not path.exists(path.join(plots_folder, "sep_cls_study")):
    os.mkdir(path.join(plots_folder, "sep_cls_study"))

print(f">> the plots location : {graph_folder}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
>> the git rep : C:\Users\loicf\Documents\IRISA\stage_4_gm\stage_4_gm
>> the plots location : C:\Users\loicf\Documents\IRISA\stage_4_gm\stage_4_gm\.cache\plots\sep_cls_study


In [60]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import pandas as pd
import torch
import numpy as np
import sklearn.metrics as metrics
import random

from training_bert import BertNliLight
from custom_data_set import SnliDataset
from custom_data_set import test_dir, dev_dir
from torch.utils.data import DataLoader
import pickle

# --> from this environment
from attention_algorithms.raw_attention import RawAttention
from attention_algorithms.attention_metrics import normalize_attention
from attention_algorithms.attention_metrics import default_plot_colormap
from attention_algorithms.attention_metrics import attention_score

# --> display HTML outputs
from attention_algorithms.plausibility_visu import construct_html_table
from attention_algorithms.plausibility_visu import hightlight_txt
from IPython.display import display, HTML
from ipywidgets import interact_manual

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
ckp = path.join(".cache", "logs", "igrida_trained", "0", "best.ckpt")
model = BertNliLight.load_from_checkpoint(ckp)
model = model.eval()  # make sure to have the model in eval mod before using it

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [62]:
# the data we annotated.
e_snli_data = pd.read_csv(os.path.join('.cache', 'raw_data', 'e_snli', 'cleaned_data', 'test.csv'), sep=",")\
    [["tok_sent", "hg_goal", "label"]]
e_snli_data.head()

,tok_sent,hg_goal,label
0,"['[CLS]', 'this', 'church', 'choir', 'sings', ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",neutral
1,"['[CLS]', 'this', 'church', 'choir', 'sings', ...","[0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...",entailment
2,"['[CLS]', 'this', 'church', 'choir', 'sings', ...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",contradiction
3,"['[CLS]', 'a', 'woman', 'with', 'a', 'green', ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",neutral
4,"['[CLS]', 'a', 'woman', 'with', 'a', 'green', ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",entailment


In [63]:
# load the data
data_set = SnliDataset(dir=test_dir, nb_sentences=1000, msg=False, keep_neutral=False)
data_loader = DataLoader(data_set, batch_size=1000, shuffle=False)

sentences, masks, train_labels = next(iter(data_loader))

In [64]:
pur_attention, Y_test = None, None
auc_path = path.join(plots_folder, "auc_head_map")
dir = os.path.join(auc_path, "inference_dict.pickle")

if os.path.exists(dir) and os.path.getsize(dir) != 0:
    print(">> the files already exist in the environment >> loading the files ...", end="")
    with open(os.path.join(plots_folder, "inference_dict.pickle"), "rb") as f:
        pur_attention, Y_test = pickle.load(f)
    print(" loading finished")
else :
    with open(os.path.join(os.path.join(plots_folder,"inference_dict.pickle")), "wb") as f:
        pur_attention, Y_test, _ = attention_score(sentences=sentences, masks=masks,
                                                   e_snli_data=e_snli_data,
                                                   model=model, TR_q=0,
                                                   quantiles_calc=False)

        pickle.dump([pur_attention, Y_test], f)

>> the files already exist in the environment >> loading the files ... loading finished


In [65]:
sent_choice = [random.randint(0, 1000) for _ in range(4)]
print(f">> we will look at the sentence : {sent_choice}")

>> we will look at the sentence : [507, 699, 766, 259]


In [69]:
# This time we will keep the specials tokens to see what will happen
with open(os.path.join(graph_folder,'layer_comp_annotations.html'), 'w') as f :
    for i in range(12):
        annot = []

        for k in sent_choice:
            buff_dict = {}
            sent = sentences[k, :].clone().detach()[None, :]
            mk = masks[k, :].clone().detach()[None, :]
            raw_attention_inst = RawAttention(model=model,
                                              input_ids=sent,
                                              attention_mask=mk,
                                              test_mod=False
                                              )

            # we will now earch the e-snli corresponding sentence
            j = 0

            while eval(e_snli_data["tok_sent"][j]) != raw_attention_inst.tokens:
                j += 1

            buff_dict["annotation"] = hightlight_txt(raw_attention_inst.tokens,
                                                     torch.tensor(eval(e_snli_data.hg_goal[j])))

            # construction of the attention score
            attention_score = raw_attention_inst.attention_tensor[0, i, :, 0, :].sum(dim=0)
            # noramlization of the score

            attention_score = (attention_score - attention_score.min()) / (attention_score.max() - attention_score.min())

            buff_dict[f"layer {i}"] = hightlight_txt(attention=attention_score,
                                                      tokens=raw_attention_inst.tokens)
            # normalization of the score
            attention_score = attention_score = normalize_attention(tokens=raw_attention_inst.tokens,
                                                 attention=attention_score)

            annot.append(buff_dict)

        f.write(construct_html_table(annot[0].keys(), annot))

There is something interesting to notice is that we have the attention on the [CLS] token which is gronwing as measure as we climb into the architecture. Lets have a look head by head to see the behaviour of this token, and how the CLS token is constructed in terms of attention.

For each layer we will compare the behaviour of each heads

In [ ]:
# This time we will keep the specials tokens to see what will happen
for lay in range(12):
    with open(os.path.join(graph_folder,f'layer_{lay}_head_comp.html'), 'w') as f :
        for i in range(12):
            annot = []

            for k in sent_choice:
                buff_dict = {}
                sent = sentences[k, :].clone().detach()[None, :]
                mk = masks[k, :].clone().detach()[None, :]
                raw_attention_inst = RawAttention(model=model,
                                                  input_ids=sent,
                                                  attention_mask=mk,
                                                  test_mod=False
                                                  )

                # we will now earch the e-snli corresponding sentence
                j = 0

                while eval(e_snli_data["tok_sent"][j]) != raw_attention_inst.tokens:
                    j += 1

                buff_dict["annotation"] = hightlight_txt(raw_attention_inst.tokens,
                                                         torch.tensor(eval(e_snli_data.hg_goal[j])))

                # construction of the attention score
                attention_score = raw_attention_inst.attention_tensor[0, lay, i, 0, :]
                # noramlization of the score

                attention_score = (attention_score - attention_score.min()) / (attention_score.max() - attention_score.min())

                buff_dict[f"head {i}"] = hightlight_txt(attention=attention_score,
                                                          tokens=raw_attention_inst.tokens)
                # normalization of the score
                attention_score = attention_score = normalize_attention(tokens=raw_attention_inst.tokens,
                                                     attention=attention_score)

                annot.append(buff_dict)

            f.write(construct_html_table(annot[0].keys(), annot))